In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
path_file = os.getcwd()+ '/lines/lines-aaa'
text = []
count = 0
with open(path_file ,'r') as txtfile:
  for line in txtfile.readlines():
    if count == 100000:
      break
    text.append(line.strip().lower())
    
    count +=1


In [28]:
vocab_size =  20000
max_len = 15
trunc_token = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text)
padded = pad_sequences(sequences,maxlen=max_len, truncating=trunc_token,padding = 'post')




In [29]:
word_index

{'<OOV>': 1,
 'you': 2,
 'the': 3,
 'i': 4,
 'to': 5,
 'a': 6,
 'of': 7,
 'it': 8,
 'and': 9,
 'in': 10,
 'is': 11,
 'that': 12,
 'me': 13,
 'for': 14,
 'your': 15,
 'be': 16,
 'we': 17,
 'have': 18,
 'this': 19,
 'what': 20,
 'not': 21,
 'on': 22,
 'are': 23,
 'do': 24,
 'he': 25,
 "it's": 26,
 'no': 27,
 'with': 28,
 'all': 29,
 'if': 30,
 "i'm": 31,
 "don't": 32,
 'sir': 33,
 'will': 34,
 'my': 35,
 'here': 36,
 'but': 37,
 'him': 38,
 'know': 39,
 'was': 40,
 'yes': 41,
 'get': 42,
 'go': 43,
 'there': 44,
 'up': 45,
 'one': 46,
 'good': 47,
 'like': 48,
 'at': 49,
 'can': 50,
 'they': 51,
 'right': 52,
 'as': 53,
 'so': 54,
 'about': 55,
 'now': 56,
 'see': 57,
 'well': 58,
 'come': 59,
 'us': 60,
 'just': 61,
 'how': 62,
 'from': 63,
 "that's": 64,
 "you're": 65,
 'an': 66,
 'why': 67,
 'his': 68,
 'our': 69,
 "he's": 70,
 'time': 71,
 'got': 72,
 'man': 73,
 'want': 74,
 'when': 75,
 'out': 76,
 'them': 77,
 'work': 78,
 'think': 79,
 "i'll": 80,
 'take': 81,
 'would': 82,
 'by'

In [93]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word
    
    

In [12]:
def create_word_embeddings(file_path):
    
    with open(file_path , 'r') as f:
        wordToEmbedding = {}
        wordToIndex = {}
        indexToWord = {}
        
        for line in f:
            data = line.strip().split()
            token = data[0]
            wordToEmbedding[token] = np.array(data[1:] ,dtype = np.float64)



        
        tokens = sorted(wordToEmbedding.keys())
        for idx , token in enumerate(tokens):
            idx = idx + 1 #for zero masking
            wordToIndex[token] = idx
            indexToWord[idx] = token

    return wordToEmbedding , wordToIndex , indexToWord
            
            

400000

In [56]:
wordToEmbedding , wordToIndex , indexToWord = create_word_embeddings(os.getcwd()+ '/glove.6B/glove.6B.200d.txt')

In [57]:
len(wordToIndex)

400000

In [106]:
def create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord):
    
    vocablen = 20000 #for zero masking
    embedding_dimensions = 200
    
    embeddingMatrix = np.zeros((vocablen , embedding_dimensions))
    count = 0
    for word , index in word_index.items():
        if word not in wordToEmbedding.keys():
            embeddingMatrix[index ,:] = np.random.uniform(low = 0 , high =1 ,size = (1,200))
            count +=1
        else :
            
            embeddingMatrix[index , :] = wordToEmbedding[word]
        
    embeddingLayer = Embedding(vocablen , embedding_dimensions , weights = [embeddingMatrix] , trainable = False)
    print(embeddingMatrix.shape)
    print(count)
    
    return embeddingLayer

In [107]:
embeddingLayer = create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord)

(20000, 200)
1261


In [52]:
batch_size = 50000
context = np.zeros((50000,max_len))
response = np.zeros((50000,max_len))
def get_dataset(padded):
    index = 0
    for idx in range(0,100000,2):
        context[index,:] = padded[idx]
        response[index,:] = padded[idx+1]
        index +=1

In [71]:
response = list(response.swapaxes(0,1))

In [44]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [45]:
Tx = 15
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)


In [46]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator(inputs =([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context


In [47]:
post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(20000, activation=softmax)

In [65]:

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embeddingLayer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [66]:
model = model(15, 15, 128, 128, 20000, 20000)


In [67]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [68]:
s0 = np.zeros((batch_size, 128))
c0 = np.zeros((batch_size, 128))

In [72]:
model.fit([context, s0, c0], response, epochs=1, batch_size=100)

Epoch 1/1
50000/50000 [==============================] - 303s 6ms/step - loss: 2.0970 - dense_3_loss: 0.1347 - dense_3_accuracy: 1.0000 - dense_3_accuracy_1: 1.0000 - dense_3_accuracy_2: 1.0000 - dense_3_accuracy_3: 1.0000 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 1.0000 - dense_3_accuracy_6: 1.0000 - dense_3_accuracy_7: 1.0000 - dense_3_accuracy_8: 1.0000 - dense_3_accuracy_9: 1.0000 - dense_3_accuracy_10: 1.0000 - dense_3_accuracy_11: 1.0000 - dense_3_accuracy_12: 1.0000 - dense_3_accuracy_13: 1.0000 - dense_3_accuracy_14: 1.0000


In [73]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [78]:
s = [("how are you ?")]

In [79]:
seq = tokenizer.texts_to_sequences(s)
pad = pad_sequences(seq,maxlen=max_len, truncating=trunc_token,padding = 'post')


In [92]:
pad

array([[62, 23,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [83]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])

In [91]:
pred

[array([[9.9925393e-01, 3.7133507e-08, 4.2433442e-08, ..., 4.1243378e-08,
         3.6865828e-08, 4.4459689e-08]], dtype=float32),
 array([[9.9999166e-01, 4.1429163e-10, 4.9072391e-10, ..., 4.7334625e-10,
         4.1047138e-10, 5.2063021e-10]], dtype=float32),
 array([[9.9999583e-01, 2.0797399e-10, 2.4770175e-10, ..., 2.3865129e-10,
         2.0599178e-10, 2.6330171e-10]], dtype=float32),
 array([[9.9999619e-01, 1.8913694e-10, 2.2543745e-10, ..., 2.1716527e-10,
         1.8732640e-10, 2.3969832e-10]], dtype=float32),
 array([[9.9999630e-01, 1.8671537e-10, 2.2257447e-10, ..., 2.1440284e-10,
         1.8492695e-10, 2.3666236e-10]], dtype=float32),
 array([[9.9999630e-01, 1.8639050e-10, 2.2218975e-10, ..., 2.1403185e-10,
         1.8460485e-10, 2.3625465e-10]], dtype=float32),
 array([[9.9999630e-01, 1.8634677e-10, 2.2213764e-10, ..., 2.1398205e-10,
         1.8456155e-10, 2.3620014e-10]], dtype=float32),
 array([[9.9999630e-01, 1.8634073e-10, 2.2213129e-10, ..., 2.1397471e-10,
         

In [87]:
pred[1].shape

(1, 20000)

In [96]:
len(pred)

15

In [89]:
ans = beam_search_decoder(pred)

In [90]:
ans

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -0.000803331954692799),
 ([4923, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -16.718620596124648),
 ([5255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -16.73330150660527)]

In [94]:
index_word[4923]

'trembled'

In [95]:
index_word[5255]

'sodom'

In [99]:
ask = [("what is your name")]

seq = tokenizer.texts_to_sequences(ask)
pad = pad_sequences(seq,maxlen=max_len, truncating=trunc_token,padding = 'post')



In [100]:
pad

array([[ 20,  11,  15, 268,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [101]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])

In [102]:
len(pred)

15

In [104]:
ans = beam_search_decoder(pred)

In [105]:
ans


[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -0.000803331954692799),
 ([4923, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -16.718611075720204),
 ([5255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -16.73329191112445)]